In [1]:
import pandas as pd
import time
import numpy as np
from sklearn.metrics import log_loss
from scipy.optimize import minimize

# Load Data

In [2]:
train_y = np.ravel(pd.read_csv('../input/' + 'labels_BrandenMurray.csv'))

names = ['low_0','medium_0','high_0',
        'low_1','medium_1','high_1',
        'low_2','medium_2','high_2',
        'low_3','medium_3','high_3',
        'low_4','medium_4','high_4',
        'low_5','medium_5','high_5',
        'low_6','medium_6','high_6',
        'low_7','medium_7','high_7',
        'low_8','medium_8','high_8',
        'low_9','medium_9','high_9']

data_path = "../final_sub/"
total_col = 0

In [3]:
# Keras final level 

file_train      = 'train_blend_3rdKeras_100bagging_2017-04-23-14-02' + '.csv'
file_test_mean  = 'test_blend_3rdKeras_100bagging_2017-04-23-14-02' + '.csv'


train_nn_2nd      = pd.read_csv(data_path + file_train, header = None)
test_nn_mean_2nd  = pd.read_csv(data_path + file_test_mean, header = None)

n_column = train_nn_2nd.shape[1]
total_col += n_column

train_nn_2nd.columns      = ['nn_3rd_' + x for x in names[:n_column]]
test_nn_mean_2nd.columns  = ['nn_3rd_' + x for x in names[:n_column]]

print train_nn_2nd.iloc[:5,:3]
print test_nn_mean_2nd.iloc[:5,:3]

   nn_3rd_low_0  nn_3rd_medium_0  nn_3rd_high_0
0      0.363043         0.579546       0.057411
1      0.528957         0.426771       0.044272
2      0.400082         0.549894       0.050024
3      0.884255         0.112189       0.003556
4      0.960281         0.038754       0.000966
   nn_3rd_low_0  nn_3rd_medium_0  nn_3rd_high_0
0      0.161497         0.579230       0.259273
1      0.995040         0.004560       0.000400
2      0.969870         0.028686       0.001444
3      0.103304         0.478383       0.418313
4      0.743763         0.248407       0.007830


In [4]:
# XGB 3rd level

file_train     = 'train_blend_3rdXGB_BM_200bagging_2017-04-23-08-08' + '.csv'
file_test_mean = 'test_blend_3rdXGB_BM_200bagging_2017-04-23-08-08' + '.csv'


train_xgb_2nd      = pd.read_csv(data_path + file_train, header = None)
test_xgb_mean_2nd  = pd.read_csv(data_path + file_test_mean, header = None)

n_column = train_xgb_2nd.shape[1]
total_col += n_column

train_xgb_2nd.columns = ['xgb_3rd_' + x for x in names[:n_column]]
test_xgb_mean_2nd.columns = ['xgb_3rd_' + x for x in names[:n_column]]

print train_xgb_2nd.iloc[:5,:3]
print test_xgb_mean_2nd.iloc[:5,:3]

   xgb_3rd_low_0  xgb_3rd_medium_0  xgb_3rd_high_0
0       0.346271          0.619835        0.033895
1       0.526219          0.418259        0.055523
2       0.418339          0.539467        0.042194
3       0.871000          0.124609        0.004391
4       0.954981          0.043048        0.001971
   xgb_3rd_low_0  xgb_3rd_medium_0  xgb_3rd_high_0
0       0.124955          0.634827        0.240217
1       0.996138          0.003115        0.000746
2       0.960372          0.037904        0.001724
3       0.105266          0.518225        0.376509
4       0.762781          0.228698        0.008521


In [5]:
# ADET 3rd level

file_train     = 'train_blend_3rdADET_300bagging_last_2017-04-23-16-04' + '.csv'
file_test_mean = 'test_blend_3rdADET_300bagging_last_2017-04-23-16-04' + '.csv'


train_ADET_2nd      = pd.read_csv(data_path + file_train, header = None)
test_ADET_mean_2nd  = pd.read_csv(data_path + file_test_mean, header = None)

n_column = train_ADET_2nd.shape[1]
total_col += n_column

train_ADET_2nd.columns = ['ADET_3rd_' + x for x in names[:n_column]]
test_ADET_mean_2nd.columns = ['ADET_3rd_' + x for x in names[:n_column]]

print train_ADET_2nd.iloc[:5,:3]
print test_ADET_mean_2nd.iloc[:5,:3]

   ADET_3rd_low_0  ADET_3rd_medium_0  ADET_3rd_high_0
0        0.350119           0.613581         0.036300
1        0.455561           0.492156         0.052283
2        0.391325           0.557150         0.051525
3        0.781089           0.216103         0.002808
4        0.931792           0.066592         0.001617
   ADET_3rd_low_0  ADET_3rd_medium_0  ADET_3rd_high_0
0        0.101042           0.656203         0.242754
1        0.987466           0.012196         0.000338
2        0.933457           0.066180         0.000363
3        0.121048           0.438958         0.439994
4        0.754410           0.228334         0.017256


In [6]:
test_ADET_mean_2nd.shape

(74659, 3)

In [7]:
def myfunc(x):
    tmp1 = x[0]*(np.power(train_xgb_2nd.values,x[1]) * np.power(train_nn_2nd.values,1-x[1])) + (1-x[0])*train_ADET_2nd.values
    tmp = log_loss(train_y,tmp1)
#     print x,tmp
    return tmp

In [8]:
myfunc([0.5,0.5])

0.48551083242414655

In [9]:
def myfunc2(x):
    tmp1 = x[0]*(np.power(train_ADET_2nd.values,x[1]) * np.power(train_nn_2nd.values,1-x[1])) + (1-x[0])*train_xgb_2nd.values
    tmp = log_loss(train_y,tmp1)
#     print x,tmp
    return tmp

In [10]:
myfunc2([0.5,0.5])

0.48520119088480584

In [11]:
def myfunc3(x):
    tmp1 = x[0]*(np.power(train_ADET_2nd.values,x[1]) * np.power(train_xgb_2nd.values,1-x[1])) + (1-x[0])*train_nn_2nd.values
    tmp = log_loss(train_y,tmp1)
#     print x,tmp
    return tmp

In [12]:
myfunc3([0.5,0.5])

0.4853608724329862

In [13]:
def myfunc4(x):
    tmp1 = np.power(train_ADET_2nd.values,x[0])\
                * np.power(train_xgb_2nd.values,x[1])\
                * np.power(train_nn_2nd.values,1-x[0]-x[1])
    tmp = log_loss(train_y,tmp1)
#     print x,tmp
    return tmp

In [14]:
myfunc4([0.33,0.33])

0.4850967845639092

In [15]:
def myfunc5(x):
    tmp1 = x[0]*(train_ADET_2nd.values)\
            + x[1] * (train_xgb_2nd.values)\
            + (1-x[0]-x[1]) * (train_nn_2nd.values)
    tmp = log_loss(train_y,tmp1)
#     print x,tmp
    return tmp

In [16]:
myfunc5([0.33,0.33])

0.48535020906318477

In [17]:
def myfunc6(x):
    tmp1 = np.power(x[0]*(train_ADET_2nd.values) + (1-x[0]) * (train_xgb_2nd.values),x[1]) * np.power(train_nn_2nd.values,1-x[1])
    tmp = log_loss(train_y,tmp1)
#     print x,tmp
    return tmp

In [18]:
myfunc6([0.5,0.5])

0.48522642154673329

In [19]:
def myfunc7(x):
    tmp1 = np.power(x[0]*(train_nn_2nd.values) + (1-x[0]) * (train_xgb_2nd.values),x[1]) * np.power(train_ADET_2nd.values,1-x[1])
    tmp = log_loss(train_y,tmp1)
#     print x,tmp
    return tmp

In [20]:
myfunc7([0.5,0.5])

0.48534579230178421

In [21]:
def myfunc8(x):
    tmp1 = np.power(x[0]*(train_ADET_2nd.values) + (1-x[0]) * (train_nn_2nd.values),x[1]) * np.power(train_xgb_2nd.values,1-x[1])
    tmp = log_loss(train_y,tmp1)
#     print x,tmp
    return tmp 

In [22]:
myfunc8([0.5,0.5])

0.48531249916828989

In [51]:
best  = 1
for i in range(0,100,5):
    for j in range(0,100,5):
        score = myfunc8([i/100.0,j/100.0])
        if score < best:
            eff = [i/100.0,j/100.0]
            best = score
print eff
print best

[0.45, 0.6]
0.485279888496


In [47]:
myfunc4([0.32, 0.3])

0.48509164243712427

In [45]:
myfunc6([0.54, 0.6])

0.48516555272599171

In [50]:
myfunc7([0.58, 0.66])

0.48515013760833797

In [23]:
x = [0.32, 0.3]

sub = np.power(test_ADET_mean_2nd.values,x[0])\
                * np.power(test_xgb_mean_2nd.values,x[1])\
                * np.power(test_nn_mean_2nd.values,1-x[0]-x[1])
sub = pd.DataFrame(sub)
sub.columns = ["low", "medium", "high"]

sum_sub = sub.sum(axis=1)
for col in ["low", "medium", "high"]:
    sub[col] = sub[col] / sum_sub
    
    
# test_ADET_mean_2nd
# test_nn_mean_2nd
# test_xgb_mean_2nd

In [25]:
data_path = "../input/"
test_X = pd.read_csv(data_path + 'test_BM_MB_add03052240.csv')
sub_id = test_X.listing_id.values

In [26]:
sub["listing_id"] = sub_id
sub.shape

(74659, 4)

In [27]:
sub.head()

,low,medium,high,listing_id
0,0.129156,0.621839,0.249005,7142618
1,0.993964,0.005579,0.000457,7210040
2,0.958142,0.040876,0.000982,7174566
3,0.109523,0.477700,0.412777,7191391
4,0.753549,0.236100,0.010352,7171695


In [56]:
sub_name = '../output/sub_final_blend_func4.csv'

sub.to_csv(sub_name, index=False)

In [57]:
interest_levels = ['low', 'medium', 'high']

tau = {
    'low': 0.69195995, 
    'medium': 0.23108864,
    'high': 0.07695141, 
}

def correct(df):
    y = df[interest_levels].mean()
    a = [tau[k] / y[k]  for k in interest_levels]
    print a

    def f(p):
        for k in range(len(interest_levels)):
            p[k] *= a[k]
        return p / p.sum()

    df_correct = df.copy()
    df_correct[interest_levels] = df_correct[interest_levels].apply(f, axis=1)

    y = df_correct[interest_levels].mean()
    a = [tau[k] / y[k]  for k in interest_levels]
    print a

    return df_correct

In [58]:
tmp = correct(sub)

[0.99809142246628091, 1.010054611064261, 0.98746041433586274]
[0.99956005790310365, 1.0016563820998952, 0.99899282309562842]


In [59]:
sub_name = '../output/sub_final_blend_func4_correct.csv'
tmp.to_csv(sub_name, index=False)

In [28]:
interest_levels = ['low', 'medium', 'high']

tau_train = {
    'low': 0.694683, 
    'medium': 0.227529,
    'high': 0.077788, 
}

tau_test = {
    'low': 0.69195995, 
    'medium': 0.23108864,
    'high': 0.07695141, 
}

def correct(df, train=True, verbose=False):
    if train:
        tau = tau_train
    else:
        tau = tau_test
        
    df_sum = df[interest_levels].sum(axis=1)
    df_correct = df[interest_levels].copy()
    
    if verbose:
        y = df_correct.mean()
        a = [tau[k] / y[k]  for k in interest_levels]
        print( a)
    
    for c in interest_levels:
        df_correct[c] /= df_sum

    for i in range(20):
        for c in interest_levels:
            df_correct[c] *= tau[c] / df_correct[c].mean()

        df_sum = df_correct.sum(axis=1)

        for c in interest_levels:
            df_correct[c] /= df_sum
    
    if verbose:
        y = df_correct.mean()
        a = [tau[k] / y[k]  for k in interest_levels]
        print( a)

    return df_correct

In [29]:
tmp = correct(sub,train=False, verbose=True)

[0.99809142246628091, 1.010054611064261, 0.98746041433586274]
[0.99999999940553674, 1.0000000009886372, 1.0000000023764835]


In [33]:
tmp["listing_id"] = sub_id
tmp.head()

,low,medium,high,listing_id
0,0.128285,0.626923,0.244791,7142618
1,0.993886,0.005662,0.000453,7210040
2,0.957563,0.041465,0.000972,7174566
3,0.109202,0.483451,0.407348,7191391
4,0.750965,0.238824,0.010211,7171695


In [32]:
sub.head()

,low,medium,high,listing_id
0,0.129156,0.621839,0.249005,7142618
1,0.993964,0.005579,0.000457,7210040
2,0.958142,0.040876,0.000982,7174566
3,0.109523,0.477700,0.412777,7191391
4,0.753549,0.236100,0.010352,7171695


In [34]:
sub_name = '../output/sub_final_blend_func4_correct_CPMP.csv'
tmp.to_csv(sub_name, index=False)